**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
skvideo.setFFmpegPath('C:/ffmpeg/bin')
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The goal of the function act is to return the action to use at each state. To do so, it applies a $\epsilon$-greedy strategy : at state $s_t$, the agent will either pick the optimal learned action (based on all past actions) with a probability $1-\epsilon$, or will pick a random action with probability $\epsilon$. $\epsilon$ thus represents how greedy the algorithm is during the exploration : if $\epsilon$ is too small, the agent will mostly choose the action based on what he has already learned and might not discover better actions, while if $\epsilon$ is too big, the agent will explore too much, choosing a random actions most of the time and will not use his past knowledge.
The choice of $\epsilon$ is therefor essential because it determines the tradeoff between exploration and exploitation.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train= 50 # # set small when debugging
epochs_test= 20 # # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array position corresponds to the position of the rat on the island. It has a size of (grid_size,grid_size), which is two cells larger than the island, to handle the 5x5 vision range of the rat. Its value are :
\begin{equation}
  \begin{cases}
    -1, & \text{where the rat is not allowed to go (the margins)}.\\
    0, & \text{where the rat is allowed to go (the island)}.\\
    1, & \text{where the rat is located}.
  \end{cases}
\end{equation}

The array board represents the reward value of each cell:
\begin{equation}
  \begin{cases}
    -1, & \text{for a poisonous cell}.\\
    0,5 & \text{for a cheese cell}.\\
    0, & \text{for all others cells}.
  \end{cases}
\end{equation}
When a rat go to a cell, it will receive the reward associated to the cell and the reward on the cell will be fixed to 0.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        #Random action
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    for e in range(1,epochs+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False
        
        win = 0
        lose = 0
        
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
                 
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            
         # Update stats
        score = score + win - lose
        
        print("Win/lose count {:.2f}/{:.2f}. Average score ({:.2f})"
              .format(win, lose, score/e))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random20.mp4'))

Win/lose count 9.50/9.00. Average score (0.50)
Win/lose count 6.00/7.00. Average score (-0.25)
Win/lose count 10.00/13.00. Average score (-1.17)
Win/lose count 6.50/12.00. Average score (-2.25)
Win/lose count 9.00/16.00. Average score (-3.20)
Win/lose count 10.00/8.00. Average score (-2.33)
Win/lose count 10.50/10.00. Average score (-1.93)
Win/lose count 12.00/11.00. Average score (-1.56)
Win/lose count 8.00/11.00. Average score (-1.72)
Win/lose count 12.00/8.00. Average score (-1.15)
Win/lose count 10.50/13.00. Average score (-1.27)
Win/lose count 8.50/19.00. Average score (-2.04)
Win/lose count 7.50/23.00. Average score (-3.08)
Win/lose count 10.50/17.00. Average score (-3.32)
Win/lose count 14.00/19.00. Average score (-3.43)
Win/lose count 8.00/18.00. Average score (-3.84)
Win/lose count 6.00/14.00. Average score (-4.09)
Win/lose count 14.50/12.00. Average score (-3.72)
Win/lose count 7.00/11.00. Average score (-3.74)
Win/lose count 9.50/15.00. Average score (-3.83)
Final score: -3.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer__

1) Let's show that: 
\begin{equation} 
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] 
\end{equation}

\begin{equation}
\begin{aligned}
Q^{\pi}(s, a) &=E_{P^{\pi}} [\sum_{t=0}^{\infty} \gamma^t r(s_t, a_t) | s_0=s, a_0=a] \\
&=E_{P^{\pi}} [r(s, a)+\sum_{t=1}^{\infty} \gamma^t r(s_t, a_t) | s_0=s, a_0=a] \\
&=r(s, a)+\gamma E_{P^{\pi}} [\sum_{t=1}^{\infty} \gamma^{t-1} r(s_t, a_t) | s_0=s, a_0=a] \\
&= r(s, a)+ \sum_{(a',s')} p(s_1=s', a_1=a'|s_0=s, a_0=a) \gamma E_{P^{\pi}} [\sum_{t=1}^{\infty} \gamma^{t-1} r(s_t, a_t) | s_0=s, a_0=a, s_1=s', a_1=a']\\
&= r(s, a)+ \sum_{(a',s')} p(s_1=s', a_1=a'|s_0=s, a_0=a) \gamma E_{P^{\pi}} [\sum_{t=1}^{\infty} \gamma^{t-1} r(s_t, a_t) | s_1=s', a_1=a']\\
&= r(s, a)+ \sum_{(a',s')} p(s_1=s', a_1=a'|s_0=s, a_0=a) \gamma Q^{\pi}(s', a') 
\\
&= r(s, a) + E_{(s',a')\sim p(\cdot|s,a)} [\gamma Q^{\pi}(s', a')]
\\
&= E_{(s',a')\sim p(\cdot|s,a)} [r(s, a) + \gamma Q^{\pi}(s', a')] \ 
\end{aligned}
\end{equation}

2) Now ,  let's show that: 
\begin{equation}
Q^{*}(s, a)=E_{s^{\prime} \sim \pi^{*}(. | s, a)}[r(s, a)+\gamma \max _{a^{\prime}} Q^{*}(s^{\prime}, a^{\prime})]
\end{equation}


We have:
\begin{equation}
\begin{aligned}
Q^{*}(s, a)= \max_{\pi} Q^{\pi}(s, a) = Q^{\pi^*}(s, a) \, with \, \pi^* = argmax\,  Q^{\pi}(s, a) \\
\end{aligned}
\end{equation}

For each state $s$, there exist an action a such that $\pi^*(a|s) = 1$, and $Q^{\pi^{*}}(s, a)=\max _{a}, Q^{\pi^{*}}\left(s, a^{\prime}\right)$ (otherwise
$\pi^{*}$ is not optimal). Let us denote $\pi^{*}(s)$ this action.

We thus obtain:

\begin{equation}
\begin{aligned}
Q^{*}(s, a) =& Q^{\pi^{*}}(s, a)=E_{s^{\prime} \sim \pi^{*}(. | s, a)}\left[r(s, a)+\gamma Q^{\pi^{*}}\left(s^{\prime}, \pi^{*}\left(s^{\prime}\right)\right)\right] \\
=& E_{s^{\prime} \sim \pi^{*}(. | s, a)}\left[r(s, a)+\gamma \max _{a}, Q^{\pi^{*}}\left(s^{\prime}, a^{\prime}\right)\right] \\
=& E_{s^{\prime} \sim \pi^{*}(. | s, a)}\left[r(s, a)+\gamma \max _{a}, Q^{*}\left(s^{\prime}, a^{\prime}\right)\right]
\end{aligned}
\end{equation}

3) We can estimate the Q-function by :
\begin{equation}
\begin{aligned}
Q^{estim}(s, a,\theta)= \max_{\pi} Q^{\pi}(s, a) = Q^{\pi^*}(s, a) \, with \, \pi^* = argmax\,  Q^{\pi}(s, a,\theta) \\
\end{aligned}
\end{equation}

Then by using a quadratic loss function :
\begin{equation} 
\begin{aligned} 
\mathcal{L}(\theta) &= ||Q^{estim}(s, a, \theta) - Q(s, a, \theta)||^2 \\
&= ||\mathbb{E}{s^\prime\sim p(\cdot |s, a)} \left[r(s, a) + \gamma \max{a^\prime} Q (s^\prime, a^\prime, \theta) \right] - Q(s, a, \theta)||^2 \\ &= \mathbb{E}{s^\prime\sim p(\cdot |s, a)} \left[ ||r(s, a) + \gamma \max{a^\prime} Q (s^\prime, a^\prime, \theta) - Q(s, a, \theta)||^2\right]
\end{aligned}
\end{equation}



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1,epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
    
        while not game_over:
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.2f}/{:.2f} ({:.2f})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s,0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            s_, n_s, a_, r_, game_over_ = self.memory.random_access()
            
            input_states[i] = s_
        
            target_q[i] = self.model.predict(np.expand_dims(s_,0))
            
            if game_over_:
                
                target_q[i,a_] = r_
            else:
               
                target_q[i, a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s,0)))
       
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(32, activation="relu"))
        model.add(Dense(16, activation="relu"))
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train50.mp4'))

Epoch 001/050 | Loss 0.0126 | Win/lose count 4.00/8.00 (-4.00)
Epoch 002/050 | Loss 0.0052 | Win/lose count 10.00/9.00 (1.00)
Epoch 003/050 | Loss 0.0121 | Win/lose count 7.00/3.00 (4.00)
Epoch 004/050 | Loss 0.0093 | Win/lose count 4.50/9.00 (-4.50)
Epoch 005/050 | Loss 0.0071 | Win/lose count 2.50/4.00 (-1.50)
Epoch 006/050 | Loss 0.0119 | Win/lose count 4.00/5.00 (-1.00)
Epoch 007/050 | Loss 0.0115 | Win/lose count 4.50/3.00 (1.50)
Epoch 008/050 | Loss 0.0056 | Win/lose count 6.50/3.00 (3.50)
Epoch 009/050 | Loss 0.0129 | Win/lose count 5.50/6.00 (-0.50)
Epoch 010/050 | Loss 0.0148 | Win/lose count 6.00/2.00 (4.00)
Epoch 011/050 | Loss 0.0579 | Win/lose count 2.50/3.00 (-0.50)
Epoch 012/050 | Loss 0.0062 | Win/lose count 3.50/4.00 (-0.50)
Epoch 013/050 | Loss 0.0092 | Win/lose count 5.50/6.00 (-0.50)
Epoch 014/050 | Loss 0.0043 | Win/lose count 4.00/2.00 (2.00)
Epoch 015/050 | Loss 0.0057 | Win/lose count 1.50/2.00 (-0.50)
Epoch 016/050 | Loss 0.0048 | Win/lose count 5.50/2.00 (3.50

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(16, kernel_size=(3, 3), input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train50.mp4'))

Epoch 001/050 | Loss 0.0017 | Win/lose count 2.00/5.00 (-3.00)
Epoch 002/050 | Loss 0.0012 | Win/lose count 4.00/2.00 (2.00)
Epoch 003/050 | Loss 0.0194 | Win/lose count 3.50/4.00 (-0.50)
Epoch 004/050 | Loss 0.0014 | Win/lose count 2.50/3.00 (-0.50)
Epoch 005/050 | Loss 0.0012 | Win/lose count 2.50/5.00 (-2.50)
Epoch 006/050 | Loss 0.0028 | Win/lose count 5.00/2.00 (3.00)
Epoch 007/050 | Loss 0.0024 | Win/lose count 4.50/6.00 (-1.50)
Epoch 008/050 | Loss 0.0486 | Win/lose count 2.00/7.00 (-5.00)
Epoch 009/050 | Loss 0.0030 | Win/lose count 3.50/3.00 (0.50)
Epoch 010/050 | Loss 0.0031 | Win/lose count 3.00/2.00 (1.00)
Epoch 011/050 | Loss 0.0048 | Win/lose count 5.50/4.00 (1.50)
Epoch 012/050 | Loss 0.0028 | Win/lose count 4.50/2.00 (2.50)
Epoch 013/050 | Loss 0.0109 | Win/lose count 6.50/8.00 (-1.50)
Epoch 014/050 | Loss 0.0036 | Win/lose count 6.00/0.00 (6.00)
Epoch 015/050 | Loss 0.0011 | Win/lose count 6.50/2.00 (4.50)
Epoch 016/050 | Loss 0.0019 | Win/lose count 1.00/1.00 (0.00)
E

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 9.00/1.00. Average score (8.00)
Win/lose count 7.50/1.00. Average score (7.25)
Win/lose count 9.00/2.00. Average score (7.17)
Win/lose count 6.00/2.00. Average score (6.38)
Win/lose count 3.50/1.00. Average score (5.60)
Win/lose count 3.50/1.00. Average score (5.08)
Win/lose count 7.00/2.00. Average score (5.07)
Win/lose count 4.50/3.00. Average score (4.62)
Win/lose count 2.50/2.00. Average score (4.17)
Win/lose count 6.50/1.00. Average score (4.30)
Win/lose count 5.00/1.00. Average score (4.27)
Win/lose count 3.50/2.00. Average score (4.04)
Win/lose count 6.50/2.00. Average score (4.08)
Win/lose count 3.00/1.00. Average score (3.93)
Win/lose count 8.00/2.00. Average score (4.07)
Win/lose count 7.00/3.00. Average score (4.06)
Win/lose count 6.50/0.00. Average score (4.21)
Win/lose count 6.00/2.00. Average score (4.19)
Win/lose count 6.00/3.00. Average score (4.13)
Win/lose count 2.50/0.00. Average score (4.05)
Final score: 4.05
Test of the FC
Win/lose co

In [16]:
HTML(display_videos('cnn_test20.mp4'))

In [17]:
HTML(display_videos('fc_test20.mp4'))

We observe here that the CNN tends to largely outperforms the FC algorithm. However in both case, the agent seems to often be stuck in the same space, and do not explore the entirety of the island, preventing him from getting further bonuses and thus limiting its score.

The temperature parameter corresponds to the probability of generating a cheese or a poison at each cell of the island. If the temperature is very low, only a few cells have bonuses and maluses, therefor it tends to limit the agent's score because of the limited number of bonuses and the difficulty to learn the best actions (most actions being to go from an empty cell to another). On the other hand, if the temperature is high, there are a lot more of bonuses cells, as if there is a bonus and a malus on the cell, the cell will be considered as a bonus. The agent can therefor both win a lot and learn a lot which improve the overall score for both algorithms.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [18]:
def train_explore(agent,env,epoch,decay = 0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1,epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
            # decay the epsilon greedy parameter
            agent.set_epsilon(agent.epsilon * decay)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.2f}/{:.2f} ({:.2f})"
              .format(e, epoch, loss, win, lose, win-lose))
        
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward -= self.malus_position[self.x, self.y]  
        self.malus_position[self.x, self.y] = 0.1
        
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        
        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        # reset the malus_position to all zeros
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1), 
                               self.board.reshape(self.grid_size, self.grid_size,1), 
                        self.position.reshape(self.grid_size, self.grid_size,1)), axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    

In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, decay = 0.95, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore50.mp4'))

Epoch 001/050 | Loss 0.0053 | Win/lose count 4.50/20.70 (-16.20)
Epoch 002/050 | Loss 0.0095 | Win/lose count 1.50/20.90 (-19.40)
Epoch 003/050 | Loss 0.0092 | Win/lose count 6.00/20.20 (-14.20)
Epoch 004/050 | Loss 0.0093 | Win/lose count 7.00/20.40 (-13.40)
Epoch 005/050 | Loss 0.0244 | Win/lose count 6.50/19.40 (-12.90)
Epoch 006/050 | Loss 0.0086 | Win/lose count 3.50/20.40 (-16.90)
Epoch 007/050 | Loss 0.0040 | Win/lose count 3.50/19.40 (-15.90)
Epoch 008/050 | Loss 0.0058 | Win/lose count 7.00/17.70 (-10.70)
Epoch 009/050 | Loss 0.0041 | Win/lose count 7.00/18.10 (-11.10)
Epoch 010/050 | Loss 0.0016 | Win/lose count 2.00/20.00 (-18.00)
Epoch 011/050 | Loss 0.0126 | Win/lose count 13.00/15.70 (-2.70)
Epoch 012/050 | Loss 0.0048 | Win/lose count 9.50/16.00 (-6.50)
Epoch 013/050 | Loss 0.0086 | Win/lose count 6.00/17.90 (-11.90)
Epoch 014/050 | Loss 0.0073 | Win/lose count 1.00/20.30 (-19.30)
Epoch 015/050 | Loss 0.0042 | Win/lose count 8.00/18.40 (-10.40)
Epoch 016/050 | Loss 0.004

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore20.mp4'))

Win/lose count 6.50/0.00. Average score (6.50)
Win/lose count 12.00/0.00. Average score (9.25)
Win/lose count 11.00/0.00. Average score (9.83)
Win/lose count 8.50/0.00. Average score (9.50)
Win/lose count 22.50/1.00. Average score (11.90)
Win/lose count 15.50/1.00. Average score (12.33)
Win/lose count 26.00/0.00. Average score (14.29)
Win/lose count 16.50/0.00. Average score (14.56)
Win/lose count 27.50/0.00. Average score (16.00)
Win/lose count 18.00/0.00. Average score (16.20)
Win/lose count 20.00/2.00. Average score (16.36)
Win/lose count 18.00/2.00. Average score (16.33)
Win/lose count 24.00/0.00. Average score (16.92)
Win/lose count 24.50/2.00. Average score (17.32)
Win/lose count 16.00/1.00. Average score (17.17)
Win/lose count 17.50/1.00. Average score (17.12)
Win/lose count 12.50/0.00. Average score (16.85)
Win/lose count 17.50/0.00. Average score (16.89)
Win/lose count 19.00/0.00. Average score (17.00)
Win/lose count 20.50/0.00. Average score (17.18)
Final score: 17.175


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***